In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [3]:
df = pd.read_csv('../data/data.tsv', sep='\t',error_bad_lines=False)

b'Skipping line 83032: expected 6 fields, saw 7\n'
b'Skipping line 154657: expected 6 fields, saw 7\n'
b'Skipping line 323916: expected 6 fields, saw 7\n'
/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,133273,213221,213222.0,How is the life of a math student? Could you d...,Which level of prepration is enough for the ex...,0.0
1,402555,536040,536041.0,How do I control my horny emotions?,How do you control your horniness?,1.0
2,360472,364011,490273.0,What causes stool color to change to yellow?,What can cause stool to come out as little balls?,0.0
3,150662,155721,7256.0,What can one do after MBBS?,What do i do after my MBBS ?,1.0
4,183004,279958,279959.0,Where can I find a power outlet for my laptop ...,"Would a second airport in Sydney, Australia be...",0.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363192 entries, 0 to 363191
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            363192 non-null  object 
 1   qid1          363192 non-null  object 
 2   qid2          363185 non-null  float64
 3   question1     363181 non-null  object 
 4   question2     363180 non-null  object 
 5   is_duplicate  363180 non-null  float64
dtypes: float64(2), object(4)
memory usage: 16.6+ MB


In [6]:
df = df.dropna(axis=0, subset=('question1','question2','is_duplicate' ))

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 363177 entries, 0 to 363191
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            363177 non-null  object 
 1   qid1          363177 non-null  object 
 2   qid2          363177 non-null  float64
 3   question1     363177 non-null  object 
 4   question2     363177 non-null  object 
 5   is_duplicate  363177 non-null  float64
dtypes: float64(2), object(4)
memory usage: 19.4+ MB


In [8]:
#lower casing
def lower_case(text):
    return text.lower()


#remove numbers
def remove_numbers(text):
    output = re.sub(r'\d+', '', text)
    return output

# remove punctuation
import string
def remove_punctuation(text):
    text_p = "".join([char for char in text if char not in string.punctuation])
    return text_p

#tokenize text

import nltk
nltk.download('punkt')
from nltk import word_tokenize

def tokenize(text):
    words = word_tokenize(text)
    return words

#remove stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

def remove_stopwords(text):
    filtered_words = [word for word in text if word not in stop_words]
    return filtered_words

#stemming
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def stem_words(text):
    stemmed = [stemmer.stem(word) for word in text]
    return stemmed

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
class Preprocessor():
    def __init__(self):
        pass
    
    def preprocess(self,df_column, steps):
        self.df_column = df_column
        self.steps = steps
    
        if 'lower_case' in self.steps:
            self.df_column = self.df_column.apply(lambda x: lower_case(x))
        
        
        if 'remove_numbers' in self.steps:
            self.df_column = self.df_column.apply(lambda x: remove_numbers(x))
        
        if 'remove_punctuation' in self.steps:
            self.df_column = self.df_column.apply(lambda x: remove_punctuation(x))           
        
        if 'tokenize' in self.steps:
            self.df_column = self.df_column.apply(lambda x: tokenize(x))
        
        if 'stopwords' in self.steps:
            self.df_column = self.df_column.apply(lambda x: remove_stopwords(x))
        
        if 'stemming' in self.steps:
            self.df_column = self.df_column.apply(lambda x: stem_words(x))
            
        return self.df_column

In [10]:
steps = ['lower_case','remove_numbers','remove_punctuation',
        'tokenize','stopwords','stemming']
processor = Preprocessor()
df['question1'] = processor.preprocess(df['question1'] ,steps)

In [11]:
df['question2'] = processor.preprocess(df['question2'] ,steps)

In [12]:
df.reset_index(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363177 entries, 0 to 363176
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   index         363177 non-null  int64  
 1   id            363177 non-null  object 
 2   qid1          363177 non-null  object 
 3   qid2          363177 non-null  float64
 4   question1     363177 non-null  object 
 5   question2     363177 non-null  object 
 6   is_duplicate  363177 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 19.4+ MB


In [13]:
df.head()

,index,id,qid1,qid2,question1,question2,is_duplicate
0,0,133273,213221,213222.0,"[life, math, student, could, describ, experi]","[level, preprat, enough, exam, jlpt]",0.0
1,1,402555,536040,536041.0,"[control, horni, emot]","[control, horni]",1.0
2,2,360472,364011,490273.0,"[caus, stool, color, chang, yellow]","[caus, stool, come, littl, ball]",0.0
3,3,150662,155721,7256.0,"[one, mbb]",[mbb],1.0
4,4,183004,279958,279959.0,"[find, power, outlet, laptop, melbourn, airport]","[would, second, airport, sydney, australia, ne...",0.0


In [14]:
q2vocab = df['question2'].tolist()
q2vocab = [item for sublist in q2vocab for item in sublist]
q2vocab = list(set(q2vocab))
len(q2vocab)

50127

In [26]:
#define function 
def get_Doc_ids(vocabulary_list,dataframe,tokens_col_name, id_col_name):
    Doc_ids = {}      #initialise empty document frequency dictionary
    
    #for each word in vocabulary, create key:value pair, with an empty list as value
    for word in vocabulary_list:
        Doc_ids[word] = [] 
        
    #for each word in DF, add the id of question2 where it appears. Create a set to avoid duplicate ids
    for i in range(len(dataframe)):
        for word in dataframe.loc[i,tokens_col_name]:
            Doc_ids[word].append(dataframe[id_col_name][i])
    for word in Doc_ids.keys():
        Doc_ids[word] = list(set(Doc_ids[word]))
        #DF[word]  =len(DF[word])
        
    return Doc_ids

Doc_ids = get_Doc_ids(q2vocab,df,'question2','qid2')
list(Doc_ids.items())[:5]

[('don´t',
  [213351.0, 149992.0, 219402.0, 282252.0, 232435.0, 189494.0, 403996.0]),
 ('lever',
  [428128.0,
   449762.0,
   531365.0,
   292328.0,
   378861.0,
   508814.0,
   62577.0,
   366833.0,
   321589.0,
   371415.0,
   211835.0]),
 ('autx',
  [203426.0, 19306.0, 40138.0, 465457.0, 263411.0, 384153.0, 264572.0]),
 ('polygamist', [203268.0, 483806.0]),
 ('histon', [110980.0, 148970.0, 329847.0, 15960.0, 279005.0])]

In [41]:
#dictionary - word: list of doc_ids
Word_Doc_ids = {}
for word in q2vocab:
    Word_Doc_ids[word] = []
for i in range(len(df)):
    for word in df.loc[i,'question2']:
        Word_Doc_ids[word].append(df.loc[i,'qid2'])

#create set to drop duplicate values of doc_ids
for word in q2vocab:
    Word_Doc_ids[word] = list(set(Word_Doc_ids[word]))
# for i in range(len(q2vocab)):
#     if q2vocab[i] in df.loc[i,'question2']:
#         Word_Doc_ids[q2vocab[i]].append(df.loc[i,'qid2'])
# for word in q2vocab:        
#     Word_Doc_ids[word] = list(set(Word_Doc_ids[word]))        
    

In [42]:
list(Word_Doc_ids.items())[:2]

[('don´t',
  [213351.0, 149992.0, 219402.0, 282252.0, 232435.0, 189494.0, 403996.0]),
 ('lever',
  [428128.0,
   449762.0,
   531365.0,
   292328.0,
   378861.0,
   508814.0,
   62577.0,
   366833.0,
   321589.0,
   371415.0,
   211835.0])]

In [49]:
#create Document_frequency (IDF) dictionary
N = df['qid2'].nunique()
print(N)

IDF = {}
for word in Doc_ids.keys():
    IDF[word] = N/(len(Word_Doc_ids[word])+1)
list(IDF.items())[:5]    

273121


[('don´t', 34140.125),
 ('lever', 22760.083333333332),
 ('autx', 34140.125),
 ('polygamist', 91040.33333333333),
 ('histon', 45520.166666666664)]

In [17]:
df['q2_len'] = df['question2'].apply(lambda x: len(x) )
df.head(2)

,index,id,qid1,qid2,question1,question2,is_duplicate,q2_len
0,0,133273,213221,213222.0,"[life, math, student, could, describ, experi]","[level, preprat, enough, exam, jlpt]",0.0,5
1,1,402555,536040,536041.0,"[control, horni, emot]","[control, horni]",1.0,2


In [24]:
#dictionary - doc_id: words
doc_words = dict(zip(df['qid2'],df['question2']))
# tf_lists = {}
# for word in q2vocab:
#     word_tf = []
#     for doc in Doc_ids[word]:
#         tf = doc_words[doc].count(word)/len(doc_words[doc])
#         word_tf.append(tf)
#     tf_lists[word] = word_tf 
    
# list(tf_lists.items())[:5] 
list(doc_words.items())[:2]

[(213222.0, ['level', 'preprat', 'enough', 'exam', 'jlpt']),
 (536041.0, ['control', 'horni'])]

In [56]:
# Create a dictionary where keys are words in the vocab, 
#values are dictinary of key:values as doc_id:tf for corresponding word
#nested dictionary - word: {{doc_id1: tf}, {doc_id2: tf}, ....}
inverted_tf = {}
for word in q2vocab:
    word_tf = {}
    for doc in Word_Doc_ids[word]:
        tf = doc_words[doc].count(word)/len(doc_words[doc])
        word_tf[doc] = tf
    inverted_tf[word] = word_tf   

In [55]:
list(inverted_tf.items())[0]

('don´t',
 {213351.0: 0.09090909090909091,
  149992.0: 0.1,
  219402.0: 0.25,
  282252.0: 0.2,
  232435.0: 0.16666666666666666,
  189494.0: 0.1,
  403996.0: 0.08333333333333333})

In [20]:
# tfidf_first = []
# first = list(inverted_tf['header'].items()) 
# for tup in first:
#     tup = list(tup)
#     tfidf = tup[1]*np.log10(IDF['header'])
#     tup[1] = tfidf
#     tfidf_first.append(tup)
# tfidf_first

In [83]:
# Create a dictionary where keys are words in the vocab, 
#values are dictinary of key:values as doc_id:tfidf for corresponding word
#nested dictionary - word: {{doc_id1: tf}, {doc_id2: tf}, ....}
inverted_tfidf = {}
for word in q2vocab:
    doc_tfidfs = []
    for tup in list(inverted_tf[word].items()):
        tup = list(tup)
        tfidf = tup[1]*np.log10(IDF[word])
        tup[1] = tfidf
        doc_tfidfs.append(tup)
    inverted_tfidf[word] = doc_tfidfs  

# for i in range(len(inverted_tfidf)):
#     inverted_tfidf[q2vocab[i]] = [dict([[x[0],x[1]]])
#                                 for x in inverted_tfidf[q2vocab[i]]]
     

In [84]:
list(inverted_tfidf.items())[0]

('don´t',
 [[213351.0, 0.4121150097181024],
  [149992.0, 0.45332651068991264],
  [219402.0, 1.1333162767247815],
  [282252.0, 0.9066530213798253],
  [232435.0, 0.7555441844831876],
  [189494.0, 0.45332651068991264],
  [403996.0, 0.3777720922415938]])

In [85]:
for i in range(len(inverted_tfidf)):
    inverted_tfidf[q2vocab[i]] = {x[0]:x[1] for x in inverted_tfidf[q2vocab[i]]}

In [86]:
list(inverted_tfidf.items())[0]

('don´t',
 {213351.0: 0.4121150097181024,
  149992.0: 0.45332651068991264,
  219402.0: 1.1333162767247815,
  282252.0: 0.9066530213798253,
  232435.0: 0.7555441844831876,
  189494.0: 0.45332651068991264,
  403996.0: 0.3777720922415938})

In [126]:
queries = df[df['is_duplicate'] == 1][:100]
queries = queries[['question1','qid2','question2', 'is_duplicate']]
queries = queries.reset_index()
queries.head()

,index,question1,qid2,question2,is_duplicate
0,1,"[control, horni, emot]",536041.0,"[control, horni]",1.0
1,3,"[one, mbb]",7256.0,[mbb],1.0
2,7,"[best, self, help, book, read, chang, life]",787.0,"[top, self, help, book, read]",1.0
3,11,"[hillari, clinton, polici, toward, india, beco...",177376.0,"[hilari, clinton, polici, toward, india, becom...",1.0
4,13,"[best, book, studi, tensor, gener, rel, basic]",22333.0,"[best, book, tensor, calculu]",1.0


In [127]:
len(queries[:100])

100

In [128]:
from collections import Counter
def ranked_docs(query, inverted_file):
    counter_objects = []
    for word in query:
        counter_objects.append(Counter(inverted_file[word]))
        combined = sum(counter_objects,Counter())
        ranked_docs = sorted(combined, key = combined.get, reverse=True)
    return ranked_docs 
query = queries.loc[0,'question1']
ranked_docs(query,inverted_tfidf)

[355257.0,
 344642.0,
 536041.0,
 317944.0,
 348571.0,
 42047.0,
 132587.0,
 24179.0,
 59166.0,
 164015.0,
 231718.0,
 324584.0,
 298674.0,
 381123.0,
 8553.0,
 8554.0,
 15044.0,
 76887.0,
 5298.0,
 42275.0,
 24180.0,
 5791.0,
 71398.0,
 26547.0,
 4072.0,
 94220.0,
 268342.0,
 118928.0,
 118929.0,
 116931.0,
 479454.0,
 532816.0,
 139721.0,
 178783.0,
 179064.0,
 260988.0,
 510867.0,
 220085.0,
 320614.0,
 19607.0,
 437567.0,
 437568.0,
 339338.0,
 366214.0,
 108392.0,
 215383.0,
 16786.0,
 172437.0,
 133641.0,
 109187.0,
 58271.0,
 76916.0,
 27891.0,
 48501.0,
 138886.0,
 112916.0,
 184859.0,
 61092.0,
 61093.0,
 288549.0,
 340906.0,
 495684.0,
 433104.0,
 73188.0,
 174840.0,
 132477.0,
 407604.0,
 32868.0,
 276591.0,
 491646.0,
 456853.0,
 82166.0,
 129312.0,
 340264.0,
 469439.0,
 242115.0,
 506367.0,
 49725.0,
 102994.0,
 367189.0,
 424534.0,
 270966.0,
 82663.0,
 393969.0,
 37644.0,
 183061.0,
 258840.0,
 164663.0,
 111468.0,
 117618.0,
 537592.0,
 381970.0,
 121882.0,
 275534.0,


In [129]:
for i in range(len(queries)):
    query = queries.loc[i,'question1']
    ranked = ranked_docs(query, inverted_tfidf)
    if queries.loc[i,'qid2'] in ranked[:2]:
        queries.loc[i,'top5_pred'] = 1.0
        queries.loc[i,'top2_pred'] = 1.0
    elif queries.loc[i,'qid2'] in ranked[:5]:
        queries.loc[i,'top5_pred'] = 1.0
        queries.loc[i,'top2_pred'] = 0.0
    else:
        queries.loc[i,'top5_pred'] = 0.0
        queries.loc[i,'top2_pred'] = 0.0
    
    
queries.head()

,index,question1,qid2,question2,is_duplicate,top5_pred,top2_pred
0,1,"[control, horni, emot]",536041.0,"[control, horni]",1.0,1.0,0.0
1,3,"[one, mbb]",7256.0,[mbb],1.0,1.0,0.0
2,7,"[best, self, help, book, read, chang, life]",787.0,"[top, self, help, book, read]",1.0,0.0,0.0
3,11,"[hillari, clinton, polici, toward, india, beco...",177376.0,"[hilari, clinton, polici, toward, india, becom...",1.0,0.0,0.0
4,13,"[best, book, studi, tensor, gener, rel, basic]",22333.0,"[best, book, tensor, calculu]",1.0,0.0,0.0


In [132]:
from sklearn.metrics import accuracy_score
accuracy_score(queries['is_duplicate'],queries['top2_pred'])

0.16

In [92]:
type(merged)

collections.Counter

In [103]:
list_dicts = []
for word in query:
    list_dicts.append(Counter(inverted_tfidf[word]))
len(list_dicts)    

3

In [106]:
merged_dict = sum(list_dicts, Counter())
max(merged_dict, key=merged_dict.get)

355257.0

In [107]:
df[df['qid2'] == 355257.0]

,index,id,qid1,qid2,question1,question2,is_duplicate,q2_len
322413,322425,242931,310574,355257.0,"[alway, horni]",[horni],1.0,1


In [115]:
ranked = sorted(merged_dict, key=merged_dict.get, reverse=True)
ranked[:5]

[355257.0, 344642.0, 536041.0, 317944.0, 348571.0]